## find where the best acc 

In [ ]:
import re
import pandas as pd


In [1]:
with open('草稿.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()    # 每一行包括末尾的换行符 \n
# 去掉换行符（常见需求）
lines = [line.rstrip('\n') for line in lines]
key_lines = [line for line in lines if line.startswith('one combo')]
key_lines

['one combo training done. current idx in combos : 0, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 1, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 2, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 3, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 4, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 5, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 6, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 7, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 8, current best_val_acc : 0.09520000219345093 ',
 'one combo training done. current idx in combos : 9, current best_val_acc : 0.09520000219345093 ',


In [ ]:

# 正则说明
#   current idx in combos\s*:\s*(\d+)   -> 捕获数字 idx
#   .*?                                 -> 非贪婪匹配中间任意字符
#   best_val_acc\s*:\s*([\d.eE+-]+)     -> 捕获浮点数（支持科学计数法）
pattern = re.compile(
    r'current idx in combos\s*:\s*(\d+).*?best_val_acc\s*:\s*([\d.eE+-]+)',
    flags=re.IGNORECASE
)

# 逐行匹配，收集 (idx, best_val_acc) 元组
records = []
for line in key_lines:
    m = pattern.search(line)
    if m:
        idx = int(m.group(1))                # 索引转成 int
        val = float(m.group(2))              # 值转成 float
        records.append((idx, val))
    else:
        # 若某行不符合预期，可以打印提醒或自行决定是否抛异常
        print(f'⚠️ 解析失败: {line!r}')

In [5]:
df = pd.DataFrame(records, columns=['idx', 'best_val_acc'])
df = df.set_index('idx')        # 把 idx 设为行索引
print(df)
df.to_csv('1to10.csv')

     best_val_acc
idx              
0          0.0952
1          0.0952
2          0.0952
3          0.0952
4          0.0952
5          0.0952
6          0.0952
7          0.0952
8          0.0952
9          0.0952
10         0.0952


In [4]:
# 标记每一行是否“值改变”
df['changed'] = df['best_val_acc'].ne(df['best_val_acc'].shift())

# 第 0 行因为没有前驱，默认算作“不改变”
df.loc[0, 'changed'] = True
print(df)
changed_idx = df.index[df['changed']].tolist()
print('值发生变化的 idx →', changed_idx)

     best_val_acc  changed
idx                       
0          0.0952     True
1          0.0952    False
2          0.0952    False
3          0.0952    False
4          0.0952    False
5          0.0952    False
6          0.0952    False
7          0.0952    False
8          0.0952    False
9          0.0952    False
10         0.0952    False
值发生变化的 idx → [0]


## get combos

In [6]:
def nn_get_search_params():
    learning_rates = []
    hidden_sizes = []
    regularization_strengths = []
    learning_rate_decays = []

    learning_rates = [0.0001, 0.01, 1, 100]  # Learning rate candidates
    hidden_sizes = [2, 8, 32, 128]  # Hidden layer sizes candidates
    regularization_strengths = [0, 1e-5, 0.001, 0.1]  # Regularization strengths
    learning_rate_decays = [0.95, 0.9, 0.85]  # Learning rate decay candidates
    return (
        learning_rates,
        hidden_sizes,
        regularization_strengths,
        learning_rate_decays,
    )

In [ ]:
def nn_get_search_params():
    learning_rates = []
    hidden_sizes = []
    regularization_strengths = []
    learning_rate_decays = []

    learning_rates = [1e-3, 1e-2, 1e-4]  # Learning rate candidates
    hidden_sizes = [50, 100, 200]  # Hidden layer sizes candidates
    regularization_strengths = [1e-3, 1e-4, 1e-5]  # Regularization strengths
    learning_rate_decays = [0.95, 0.9, 0.85]  # Learning rate decay candidates

    return (
        learning_rates,
        hidden_sizes,
        regularization_strengths,
        learning_rate_decays,
    )

In [8]:
import itertools

lr, hs, reg, decay = nn_get_search_params()
combos = list(itertools.product(lr, hs, reg, decay)) 

In [10]:
for i in range(len(combos)):
    print(i, combos[i])

0 (0.001, 50, 0.001, 0.95)
1 (0.001, 50, 0.001, 0.9)
2 (0.001, 50, 0.001, 0.85)
3 (0.001, 50, 0.0001, 0.95)
4 (0.001, 50, 0.0001, 0.9)
5 (0.001, 50, 0.0001, 0.85)
6 (0.001, 50, 1e-05, 0.95)
7 (0.001, 50, 1e-05, 0.9)
8 (0.001, 50, 1e-05, 0.85)
9 (0.001, 100, 0.001, 0.95)
10 (0.001, 100, 0.001, 0.9)
11 (0.001, 100, 0.001, 0.85)
12 (0.001, 100, 0.0001, 0.95)
13 (0.001, 100, 0.0001, 0.9)
14 (0.001, 100, 0.0001, 0.85)
15 (0.001, 100, 1e-05, 0.95)
16 (0.001, 100, 1e-05, 0.9)
17 (0.001, 100, 1e-05, 0.85)
18 (0.001, 200, 0.001, 0.95)
19 (0.001, 200, 0.001, 0.9)
20 (0.001, 200, 0.001, 0.85)
21 (0.001, 200, 0.0001, 0.95)
22 (0.001, 200, 0.0001, 0.9)
23 (0.001, 200, 0.0001, 0.85)
24 (0.001, 200, 1e-05, 0.95)
25 (0.001, 200, 1e-05, 0.9)
26 (0.001, 200, 1e-05, 0.85)
27 (0.01, 50, 0.001, 0.95)
28 (0.01, 50, 0.001, 0.9)
29 (0.01, 50, 0.001, 0.85)
30 (0.01, 50, 0.0001, 0.95)
31 (0.01, 50, 0.0001, 0.9)
32 (0.01, 50, 0.0001, 0.85)
33 (0.01, 50, 1e-05, 0.95)
34 (0.01, 50, 1e-05, 0.9)
35 (0.01, 50, 1e-05,